In [1]:
import sys
import os
import subprocess

try:
    import google.colab

    ENV_IS_CL = True
    root = r"/content/t-route"
    subprocess.run(["git", 
                    "clone", 
                    "-b",
                    "courant",
                    "https://github.com/awlostowski-noaa/t-route.git"])
    
    
except:
    root = os.path.dirname(os.path.abspath("../../../../"))
    
dev_directory = os.path.join(root,"src","fortran_routing","mc_pylink_v00","MC_singleSeg_singleTS","courant_dev")
sys.path.append(dev_directory)

base_directory = os.path.join(root,"src","fortran_routing","mc_pylink_v00","MC_singleSeg_singleTS")
sys.path.append(base_directory)
    

In [2]:
# model parameters
params = {}
params["dt"] = 60.0
params["dx"] = 1800.0
params["bw"] = 112.0
params["tw"] = 248.0
params["twcc"] = 623.60
params["n"] = 0.02800000086426735
params["ncc"] = 0.03136000037193298
params["cs"] = 0.42
params["s0"] = 0.007999999690800905
params["qlat"] = 40.0
params["qup"] = 45009.0
params["quc"] = 50098.0
params["qdp"] = 50014.0
params["depthp"] = 30.0

In [3]:
# write model parameters to control file
file = open(os.path.join(dev_directory,"control.txt"), "w")
for key, value in params.items():
    file.write(str(key) + " " + str(value) + "\n")
file.close()

In [5]:
# Compile fortran modules and test program, create an executable
# ! ./compile

# copy varPrecision.f90 from base directory
cmd = subprocess.run(
    ["cp", 
     os.path.join(base_directory, "varPrecision.f90"), 
     os.path.join(dev_directory,  "varPrecision.f90")],
    cwd=dev_directory,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# compile courant modified MC module and testing program
cmd = subprocess.run(
    ["gfortran",
     "-c",
     "varPrecision.f90",
     "MCsingleSegStime_f2py_NOLOOP-anw.f90",
     "test_MC_program.f90"],
    cwd=dev_directory,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# create executable of test program
cmd = subprocess.run(
    ["gfortran",
     "varPrecision.o",
     "MCsingleSegStime_f2py_NOLOOP-anw.o",
     "test_MC_program.o",
     "-o",
     "mc_test.exe"],
    cwd=dev_directory,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# check to make sure the compilation process worked
if os.path.exists("mc_test.exe") == True:
    print("Compilation process was successful, mc_test.exe exists")
else:
    print("WARNING: There was a problem with the compilatuion process, mc_test.exe does not exist!")

Compilation process was successful, mc_test.exe exists


In [16]:
# run the model
cmd = subprocess.run(
    ["./mc_test.exe"],
    cwd=dev_directory,
    stdout=subprocess.PIPE
)
print("Output from mc_test.exe:")
print(cmd.stdout)

Output from mc_test.exe:
b' --------------------------------------------------------------\n  \n MC model inputs\n  \n dt:    60.0000000    \n dx:    1800.00000    \n bw:    112.000000    \n tw:    248.000000    \n twcc:    623.599976    \n n:    2.80000009E-02\n ncc:    3.13600004E-02\n cs:   0.419999987    \n s0:    7.99999945E-03\n qlat:    40.0000000    \n qup:    45009.0000    \n quc:    50098.0000    \n qdp:    50014.0000    \n depthp:    30.0000000    \n --------------------------------------------------------------\n  \n MC model outputs\n  \n flow [m^3/s] =    46502.3516    \n velocity [m/s] =    17.7658043    \n depth [m] =    17.2265453    \n kinematic celerity [m/s] =    25.4746609    \n Courant number [-] =   0.849155426    \n'


In [7]:
# os.chdir(root)
# ! f2py3 -c varPrecision.f90 MCsingleSegStime_f2py_NOLOOP.f90 -m mc_sseg_stime

cmd = subprocess.run(
    ["f2py3",
     "-c",
     "varPrecision.o",
     "MCsingleSegStime_f2py_NOLOOP.o",
     "-m",
     "mc_sseg_stime"],
    cwd=base_directory,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

try:
    # import MC subroutine from shared object
    from mc_sseg_stime import muskingcunge_module
    print("f2py shared object successfully loaded")
except:
    print("WARNING: f2py shared object does not exist")

# import python function to call FORTRAN muskingcunge subroutine
from mc_sseg_stime_NOLOOP_demo import singlesegment

f2py shared object successfully loaded


In [8]:
# execute the model
qdc, velc, depthc = singlesegment(
        dt=params["dt"],
        qup=params["qup"],
        quc=params["quc"],
        qdp=params["qdp"],
        qlat=params["qlat"],
        dx=params["dx"],
        bw=params["bw"],
        tw=params["tw"],
        twcc=params["twcc"],
        n_manning=params["n"],
        n_manning_cc=params["ncc"],
        cs=params["cs"],
        s0=params["s0"],
        velp=0,
        depthp=params["depthp"],
    )

print("flow [m^3/s] = ", qdc)
print("velocity [m/s] = ", velc)
print("depth [m] = ", depthc)

flow [m^3/s] =  46502.3515625
velocity [m/s] =  17.765804290771484
depth [m] =  17.226545333862305


In [ ]:
# calculate kinematic celerity and courant number from model parameters and output from f2py process